In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import operator
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import normalize

In [135]:
df = pd.read_csv('train.csv')
df.head()

label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0      1       0       0       0       0       0       0       0       0   
1      0       0       0       0       0       0       0       0       0   
2      1       0       0       0       0       0       0       0       0   
3      4       0       0       0       0       0       0       0       0   
4      0       0       0       0       0       0       0       0       0   

   pixel8    ...     pixel774  pixel775  pixel776  pixel777  pixel778  \
0       0    ...            0         0         0         0         0   
1       0    ...            0         0         0         0         0   
2       0    ...            0         0         0         0         0   
3       0    ...            0         0         0         0         0   
4       0    ...            0         0         0         0         0   

   pixel779  pixel780  pixel781  pixel782  pixel783  
0         0         0         0         0         0  
1         0         0         0         0         0  
2         0         0         0         0         0  
3         0         0         0         0         0  
4         0         0         0         0         0  

[5 rows x 785 columns]

In [136]:
df.shape

(42000, 785)

In [137]:
X = df.iloc[:, 1:]
y = df.iloc[:, 0]

In [138]:
X = normalize(X, norm='l2')

In [139]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=3)

In [140]:
# sns.distplot(y, kde=False)

In [142]:
# The result is okey. 
# However, the training process tooks a lot longer when it comes to large datasets.
clf = SVC(gamma=1.69)
print(clf.fit(X_train, y_train))
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=1.69, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
0.978066378066


In [ ]:
# try and get best gamma=1.69
init = 0.01
gs = []
accs = []
for i in range(1, 201):
    print(i)
    g = i*init
    clf = SVC(kernel='rbf', gamma=g)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    gs.append(g)
    accs.append(acc)

In [ ]:
plt.xlabel("C")
plt.ylabel("Acc")
plt.plot(gs, accs)

In [ ]:
# gamma = 1.69
index, value = max(enumerate(accs), key=operator.itemgetter(1))
print(index, value)
print(gs[index])

In [67]:
import pandas as pd
import numpy as np
import h5py

from keras import backend as K
from keras.datasets import mnist
from keras.utils import np_utils

from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import RMSprop

from keras.utils import plot_model
from keras.utils import np_utils

import seaborn as sns
sns.set_style('darkgrid')

In [54]:
df = pd.read_csv('train.csv')

In [55]:
X = df.iloc[:,1:]
y = df.iloc[:, 0]

In [56]:
X = X.as_matrix()
X = X.reshape(-1, 28, 28, 1)

In [57]:
y = pd.get_dummies(y)
y = y.as_matrix()

In [104]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [105]:
rms = RMSprop(lr=1e-4)
model.compile(loss='categorical_crossentropy', optimizer=rms, metrics=['accuracy'])
for i in range(3):
    history = model.fit(X, y, batch_size=32, epochs=1, validation_split=0.33)
    model.save_weights('mnist_weight.h5')

Train on 28139 samples, validate on 13861 samples
Epoch 1/1
28139/28139 [==============================] - 226s - loss: 0.6324 - acc: 0.9113 - val_loss: 0.0800 - val_acc: 0.9758
Train on 28139 samples, validate on 13861 samples
Epoch 1/1
28139/28139 [==============================] - 220s - loss: 0.0525 - acc: 0.9839 - val_loss: 0.0622 - val_acc: 0.9817
Train on 28139 samples, validate on 13861 samples
Epoch 1/1
 4096/28139 [===>..........................] - ETA: 172s - loss: 0.0201 - acc: 0.9941

KeyboardInterrupt: 

In [106]:
df = pd.read_csv('test.csv')
X_test = df

In [107]:
X_test = X_test.as_matrix()
X_test = X_test.reshape(-1, 28, 28, 1)

In [119]:
model.load_weights('mnist_weight.h5')
y_pred = model.predict_classes(X_test)

28000/28000 [==============================] - 58s    


In [128]:
index = np.arange(y_pred.shape[0]) + 1

[    1     2     3 ..., 27998 27999 28000]


In [130]:
df = pd.DataFrame({'ImageId':index, 'Label':y_pred})
df.head()

ImageId  Label
0        1      2
1        2      0
2        3      9
3        4      9
4        5      3

In [131]:
df.to_csv('output.csv', index=False)

In [132]:
df = pd.read_csv('output.csv')
df.head()

ImageId  Label
0        1      2
1        2      0
2        3      9
3        4      9
4        5      3